In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(".")
sys.path.append("..")

from pathlib import Path
from src.datamodule.av2_dataset import Av2Dataset
from src.datamodule.av2_extractor import Av2Extractor
from src.datamodule.av2_dataset import collate_fn
from av2.datasets.motion_forecasting import scenario_serialization
from av2.map.map_api import ArgoverseStaticMap
from src.utils.vis import  visualize_scenario
from src.model.trainer_forecast import Trainer as Model
from av2.datasets.motion_forecasting.data_schema import (
    ArgoverseScenario,
    ObjectType,
    TrackCategory,
)
import matplotlib.pyplot as plt
import numpy as np

data_root = Path("/data/jerome.zhou/prediction_dataset/av2/forecast-sept-dev") # Path to the dataset
dataset = Av2Dataset(data_root=data_root, cached_split="val", extractor=Av2Extractor(radius=150))

In [ ]:
# use dataset get_data() api interface
data = dataset[16]
scene_id = data["scenario_id"]
scene_file = data_root / scene_id / ("scenario_" + scene_id + ".parquet")
map_file = data_root / scene_id / ("log_map_archive_" + scene_id + ".json")
scenario = scenario_serialization.load_argoverse_scenario_parquet(scene_file)
static_map = ArgoverseStaticMap.from_json(map_file)
# vis used original dataset output.
# visualize_scenario(scenario, static_map, transform=False)
visualize_scenario(scenario, static_map)

In [ ]:
data = dataset[40]
traj = data['x_trans']
mask = data['x_padding_mask']
lane = data['lane_positions']
lane_candidate = data["lane_candidate"]
lane = lane[lane_candidate.bool()]
for i in range(traj.size(0)):
    trajectory = traj[i][~mask[i]]
    if i == 0:
        plt.plot(trajectory[:,0], trajectory[:,1], c="k")
    else:
        plt.plot(trajectory[:,0], trajectory[:,1], c="y", alpha=0.4)
for j in range(lane.size(0)):
    lane_j = lane[j]
    # if lane_candidate[j]:
    plt.plot(lane_j[:,0], lane_j[:, 1], c='r')
    # else:
        # plt.plot(lane_j[:,0], lane_j[:, 1], c='grey', alpha=0.2)